# Human-in-the-Loop Workflow met Microsoft Agent Framework

## 🎯 Leerdoelen

In dit notebook leer je hoe je **human-in-the-loop** workflows implementeert met behulp van Microsoft Agent Framework's `RequestInfoExecutor`. Dit krachtige patroon stelt je in staat om AI-workflows te pauzeren om menselijke input te verzamelen, waardoor je agents interactief worden en mensen controle krijgen over cruciale beslissingen.

## 🔄 Wat is Human-in-the-Loop?

**Human-in-the-loop (HITL)** is een ontwerpprincipe waarbij AI-agents hun uitvoering pauzeren om menselijke input te vragen voordat ze doorgaan. Dit is essentieel voor:

- ✅ **Cruciale beslissingen** - Vraag menselijke goedkeuring voordat belangrijke acties worden ondernomen
- ✅ **Onduidelijke situaties** - Laat mensen verduidelijken wanneer AI onzeker is
- ✅ **Gebruikersvoorkeuren** - Vraag gebruikers om te kiezen tussen meerdere opties
- ✅ **Naleving & veiligheid** - Zorg voor menselijke controle bij gereguleerde operaties
- ✅ **Interactieve ervaringen** - Bouw conversatie-agents die reageren op gebruikersinput

## 🏗️ Hoe werkt het in Microsoft Agent Framework?

Het framework biedt drie belangrijke componenten voor HITL:

1. **`RequestInfoExecutor`** - Een speciale executor die de workflow pauzeert en een `RequestInfoEvent` genereert
2. **`RequestInfoMessage`** - Basisklasse voor getypeerde verzoekpayloads die naar mensen worden gestuurd
3. **`RequestResponse`** - Verbindt menselijke reacties met oorspronkelijke verzoeken via `request_id`

**Workflowpatroon:**
```
Agent detects need for input
    ↓
Sends message to RequestInfoExecutor
    ↓
Workflow pauses & emits RequestInfoEvent
    ↓
Application collects human input (console, UI, etc.)
    ↓
Application sends RequestResponse via send_responses_streaming()
    ↓
Workflow resumes with human input
```

## 🏨 Ons voorbeeld: Hotelreservering met gebruikersbevestiging

We bouwen voort op de conditionele workflow door menselijke bevestiging toe te voegen **voordat** alternatieve bestemmingen worden voorgesteld:

1. Gebruiker vraagt om een bestemming (bijv. "Parijs")
2. `availability_agent` controleert of er kamers beschikbaar zijn
3. **Als er geen kamers beschikbaar zijn** → `confirmation_agent` vraagt: "Wilt u alternatieven zien?"
4. Workflow **pauzeert** met behulp van `RequestInfoExecutor`
5. **Mens reageert** met "ja" of "nee" via console-input
6. `decision_manager` stuurt door op basis van de reactie:
   - **Ja** → Toon alternatieve bestemmingen
   - **Nee** → Annuleer reserveringsverzoek
7. Toon het eindresultaat

Dit laat zien hoe je gebruikers controle kunt geven over de suggesties van de agent!

---

Laten we beginnen! 🚀


## Stap 1: Vereiste bibliotheken importeren

We importeren de standaardcomponenten van het Agent Framework plus **specifieke klassen voor menselijke tussenkomst**:
- `RequestInfoExecutor` - Executor die de workflow pauzeert voor menselijke input
- `RequestInfoEvent` - Event dat wordt uitgezonden wanneer menselijke input wordt gevraagd
- `RequestInfoMessage` - Basisklasse voor getypeerde verzoekpayloads
- `RequestResponse` - Correlatie tussen menselijke reacties en verzoeken
- `WorkflowOutputEvent` - Event voor het detecteren van workflowuitvoer


In [21]:
import asyncio
import json
import os
from dataclasses import dataclass
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Executor,
    RequestInfoEvent,          # NEW: Event when human input is requested
    RequestInfoExecutor,       # NEW: Executor that gathers human input
    RequestInfoMessage,        # NEW: Base class for request payloads
    RequestResponse,           # NEW: Correlates response with request
    Role,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,       # NEW: Event for workflow outputs
    WorkflowRunState,          # NEW: Enum of workflow run states
    WorkflowStatusEvent,       # NEW: Event for run state changes
    ai_function,
    executor,
    handler,                   # NEW: Decorator for executor methods
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")
print("🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse")

✅ All imports successful!
🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse


## Stap 2: Definieer Pydantic-modellen voor gestructureerde outputs

Deze modellen definiëren het **schema** dat agents zullen retourneren. We behouden alle modellen uit de conditionele workflow en voegen toe:

**Nieuw voor Human-in-the-Loop:**
- `HumanFeedbackRequest` - Subklasse van `RequestInfoMessage` die de aanvraagpayload definieert die naar mensen wordt gestuurd
  - Bevat `prompt` (vraag om te stellen) en `destination` (context over de niet-beschikbare stad)


In [22]:
# Existing models from conditional workflow
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""
    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""
    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""
    destination: str
    action: str
    message: str


# NEW: Pydantic model for agent's response format
class ConfirmationQuestion(BaseModel):
    """
    Pydantic model used by confirmation_agent's response_format.
    This is what the agent will output as JSON.
    """
    question: str  # The question to ask the user
    destination: str  # The unavailable destination for context


# NEW: Dataclass for RequestInfoExecutor
@dataclass
class HumanFeedbackRequest(RequestInfoMessage):
    """
    Request sent to RequestInfoExecutor asking if user wants alternatives.
    
    MUST be a dataclass subclassing RequestInfoMessage for type compatibility.
    This is what gets sent to the RequestInfoExecutor.
    """
    prompt: str = ""  # The question to ask the user
    destination: str = ""  # The unavailable destination for context


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")
print("   - ConfirmationQuestion (agent response format) 🆕")
print("   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)
   - ConfirmationQuestion (agent response format) 🆕
   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕


## Stap 3: Maak de Hotelboekingshulpmiddel

Dezelfde tool als in de voorwaardelijke workflow - controleert of er kamers beschikbaar zijn op de bestemming.


In [23]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Stap 4: Definieer Voorwaardefuncties voor Routing

We hebben **vier voorwaardefuncties** nodig voor onze human-in-the-loop workflow:

**Van de conditionele workflow:**
1. `has_availability_condition` - Routeert wanneer hotels WEL beschikbaar zijn
2. `no_availability_condition` - Routeert wanneer hotels NIET beschikbaar zijn

**Nieuw voor human-in-the-loop:**
3. `user_wants_alternatives_condition` - Routeert wanneer de gebruiker "ja" zegt tegen alternatieven
4. `user_declines_alternatives_condition` - Routeert wanneer de gebruiker "nee" zegt tegen alternatieven


In [24]:
# Existing condition functions from conditional workflow
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )
        return result.has_availability
    except Exception as e:
        display(HTML(f"""<div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'><strong>⚠️  Error:</strong> {str(e)}</div>"""))
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )
        return not result.has_availability
    except Exception as e:
        return False


# NEW: Condition functions for human-in-the-loop routing
def user_wants_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user WANTS to see alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            wants_alternatives = "wants to see alternative" in msg_text or "want to see alternative" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
                    <strong>🔍 User Decision:</strong> User wants alternatives = <strong>{wants_alternatives}</strong>
                </div>
            """)
            )
            
            return wants_alternatives
    
    return False
def user_declines_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user DECLINES alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            declined = "declined" in msg_text or "has declined" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fce4ec; border-left: 4px solid #c2185b; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 User Decision:</strong> User declined alternatives = <strong>{declined}</strong>
                </div>
            """)
            )
            
            return declined
    
    return False
print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")
print("   - user_wants_alternatives_condition (routes when user says yes) 🆕")
print("   - user_declines_alternatives_condition (routes when user says no) 🆕")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)
   - user_wants_alternatives_condition (routes when user says yes) 🆕
   - user_declines_alternatives_condition (routes when user says no) 🆕


## Stap 5: Maak de Decision Manager Executor

Dit is de **kern van het human-in-the-loop patroon**! De `DecisionManager` is een aangepaste `Executor` die:

1. **Menselijke feedback ontvangt** via `RequestResponse` objecten
2. **De beslissing van de gebruiker verwerkt** (ja/nee)
3. **De workflow stuurt** door berichten naar de juiste agents te sturen

Belangrijke kenmerken:
- Gebruikt de `@handler` decorator om methoden als workflowstappen beschikbaar te maken
- Ontvangt `RequestResponse[HumanFeedbackRequest, str]` met zowel het oorspronkelijke verzoek als het antwoord van de gebruiker
- Geeft eenvoudige "ja" of "nee" berichten door die onze conditiefuncties activeren


In [25]:
class DecisionManager(Executor):
    """
    Coordinates workflow routing based on human feedback.
    
    This executor receives RequestResponse objects from the RequestInfoExecutor
    and makes routing decisions by sending simple messages that trigger
    condition functions.
    """

    def __init__(self, id: str | None = None):
        super().__init__(id=id or "decision_manager")

    @handler
    async def on_human_feedback(
        self,
        feedback: RequestResponse[HumanFeedbackRequest, str],
        ctx: WorkflowContext[AgentExecutorRequest],
    ) -> None:
        """
        Process human feedback and let the workflow route based on conditions.
        
        The RequestResponse contains:
        - feedback.data: The user's string reply (e.g., "yes" or "no")
        - feedback.original_request: The HumanFeedbackRequest with context
        
        This handler just displays feedback and passes the RequestResponse through.
        The routing is done by condition functions on the edges.
        """
        user_reply = (feedback.data or "").strip().lower()
        destination = getattr(feedback.original_request, "destination", "unknown")

        display(
            HTML(f"""
            <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
                <strong>🎯 Decision Manager:</strong> Processing user reply: <strong>"{user_reply}"</strong> for {destination}
            </div>
        """)
        )

        if user_reply == "yes":
            display(
                HTML("""
                <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                    <strong>➡️  Routing:</strong> User wants alternatives → Will route to alternative_agent
                </div>
            """)
            )
            # Create and send a message for the alternative_agent
            user_msg = ChatMessage(
                Role.USER,
                text=f"The user wants to see alternative destinations near {destination}. Please suggest one.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        elif user_reply == "no":
            display(
                HTML("""
                <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 Routing:</strong> User declined alternatives → Will route to cancellation_agent
                </div>
            """)
            )
            # Create and send a message for the cancellation_agent
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        else:
            # Handle unexpected input - treat as decline
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                    <strong>⚠️  Warning:</strong> Unexpected input "{user_reply}" - treating as decline
                </div>
            """)
            )
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))


print("✅ DecisionManager executor created with @handler method for human feedback")

✅ DecisionManager executor created with @handler method for human feedback


## Stap 6: Maak een aangepaste Display Executor

Dezelfde display executor uit de conditionele workflow - levert eindresultaten op als workflow-uitvoer.


In [26]:
@executor(id="prepare_human_request")
async def prepare_human_request(
    response: AgentExecutorResponse, 
    ctx: WorkflowContext[HumanFeedbackRequest]
) -> None:
    """
    Transform agent response into HumanFeedbackRequest for RequestInfoExecutor.
    
    This executor bridges the type gap between:
    - confirmation_agent outputs AgentExecutorResponse with ConfirmationQuestion JSON
    - request_info_executor expects HumanFeedbackRequest (RequestInfoMessage dataclass)
    """
    display(
        HTML("""
        <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Transform:</strong> Converting ConfirmationQuestion to HumanFeedbackRequest
        </div>
    """)
    )
    
    # Parse the agent's Pydantic output (ConfirmationQuestion)
    confirmation = ConfirmationQuestion.model_validate_json(response.agent_run_response.text)
    
    # Convert to HumanFeedbackRequest dataclass for RequestInfoExecutor
    feedback_request = HumanFeedbackRequest(
        prompt=confirmation.question,
        destination=confirmation.destination
    )
    
    # Send the properly typed RequestInfoMessage to the RequestInfoExecutor
    await ctx.send_message(feedback_request)


@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ prepare_human_request executor created with @executor decorator")
print("✅ display_result executor created with @executor decorator")

✅ prepare_human_request executor created with @executor decorator
✅ display_result executor created with @executor decorator


## Stap 7: Laad omgevingsvariabelen

Configureer de LLM-client (GitHub Models, Azure OpenAI of OpenAI).


In [27]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"), 
    api_key=os.environ.get("GITHUB_TOKEN"), 
    model_id="gpt-4o"
)

print("✅ Chat client configured with GitHub Models")

✅ Chat client configured with GitHub Models


## Stap 8: Maak AI-agenten en uitvoerders

We maken **zes workflowcomponenten**:

**Agenten (ingepakt in AgentExecutor):**
1. **availability_agent** - Controleert hotelbeschikbaarheid met behulp van de tool
2. **confirmation_agent** - 🆕 Bereidt het verzoek om menselijke bevestiging voor
3. **alternative_agent** - Stelt alternatieve steden voor (wanneer gebruiker ja zegt)
4. **booking_agent** - Stimuleert boeken (wanneer kamers beschikbaar zijn)
5. **cancellation_agent** - 🆕 Behandelt annuleringsbericht (wanneer gebruiker nee zegt)

**Speciale uitvoerders:**
6. **request_info_executor** - 🆕 `RequestInfoExecutor` die de workflow pauzeert voor menselijke input
7. **decision_manager** - 🆕 Aangepaste uitvoerder die routeert op basis van menselijke reactie (reeds hierboven gedefinieerd)


In [28]:
# Agent 1: Check availability with tool (same as conditional workflow)
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: NEW - Prepare human confirmation request
confirmation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user's requested destination has no available hotel rooms. "
            "Create a polite message asking if they would like to see alternative destinations nearby. "
            "Return a JSON with: destination (the unavailable city), and question (a friendly yes/no question). "
            "Keep the question concise and friendly."
        ),
        response_format=ConfirmationQuestion,  # Use Pydantic model for agent output
    ),
    id="confirmation_agent",
)

# Agent 3: Suggest alternative (when user says yes)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 4: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

# Agent 5: NEW - Handle cancellation when user declines alternatives
class CancellationMessage(BaseModel):
    """Message when user declines alternatives."""
    status: str
    message: str

cancellation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user has declined to see alternative hotel destinations. "
            "Create a polite cancellation message. "
            "Return JSON with: status (should be 'cancelled'), and message (a friendly acknowledgment). "
            "Keep the message brief and understanding."
        ),
        response_format=CancellationMessage,
    ),
    id="cancellation_agent",
)

# NEW: RequestInfoExecutor - pauses workflow to gather human input
request_info_executor = RequestInfoExecutor(id="request_info")

# NEW: DecisionManager instance - routes based on human feedback
decision_manager = DecisionManager(id="decision_manager")

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created Workflow Components:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>confirmation_agent</strong> 🆕 - Prepares human confirmation request</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
            <li><strong>cancellation_agent</strong> 🆕 - Handles user declining alternatives</li>
            <li><strong>request_info_executor</strong> 🆕 - Pauses workflow for human input</li>
            <li><strong>decision_manager</strong> 🆕 - Routes based on human response</li>
        </ul>
    </div>
""")
)

## Stap 9: Bouw de workflow met menselijke tussenkomst

Nu construeren we de workflowgrafiek met **conditionele routering**, inclusief het pad met menselijke tussenkomst:

**Workflowstructuur:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙                    ↘
[no_availability]        [has_availability]
        ↓                        ↓
confirmation_agent          booking_agent
        ↓                        ↓
prepare_human_request      display_result
        ↓
request_info_executor (PAUSE)
        ↓
decision_manager
   ↙         ↘
[yes]        [no]
   ↓           ↓
alternative  cancellation
   ↓           ↓
display_result
```

**Belangrijke verbindingen:**
- `availability_agent → confirmation_agent` (wanneer er geen kamers beschikbaar zijn)
- `confirmation_agent → prepare_human_request` (transformatietype)
- `prepare_human_request → request_info_executor` (pauze voor menselijke tussenkomst)
- `request_info_executor → decision_manager` (altijd - levert RequestResponse)
- `decision_manager → alternative_agent` (wanneer gebruiker "ja" zegt)
- `decision_manager → cancellation_agent` (wanneer gebruiker "nee" zegt)
- `availability_agent → booking_agent` (wanneer er kamers beschikbaar zijn)
- Alle paden eindigen bij `display_result`


In [29]:
# Build the workflow with human-in-the-loop routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    
    # NO AVAILABILITY PATH (with human-in-the-loop)
    .add_edge(availability_agent, confirmation_agent, condition=no_availability_condition)
    .add_edge(confirmation_agent, prepare_human_request)  # Transform to HumanFeedbackRequest
    .add_edge(prepare_human_request, request_info_executor)  # Send to RequestInfoExecutor
    .add_edge(request_info_executor, decision_manager)    # Always goes to decision manager
    
    # Decision manager routes based on user response
    .add_edge(decision_manager, alternative_agent, condition=user_wants_alternatives_condition)
    .add_edge(decision_manager, cancellation_agent, condition=user_declines_alternatives_condition)
    .add_edge(alternative_agent, display_result)
    .add_edge(cancellation_agent, display_result)
    
    # HAS AVAILABILITY PATH (no human input needed)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Human-in-the-Loop Routing:</strong><br>
            • If <strong>NO availability</strong> → confirmation_agent → prepare_human_request → request_info_executor → <strong>PAUSE FOR HUMAN</strong> → decision_manager<br>
            &nbsp;&nbsp;• If user says <strong>YES</strong> → alternative_agent → display_result<br>
            &nbsp;&nbsp;• If user says <strong>NO</strong> → cancellation_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result (no human input needed)
        </p>
    </div>
""")
)

## Stap 10: Voer Testgeval 1 uit - Stad ZONDER Beschikbaarheid (Parijs met Menselijke Bevestiging)

Deze test demonstreert de **volledige mens-in-de-lus cyclus**:

1. Vraag een hotel aan in Parijs
2. availability_agent controleert → Geen kamers beschikbaar
3. confirmation_agent maakt een vraag gericht op de mens
4. request_info_executor **pauzeert de workflow** en genereert `RequestInfoEvent`
5. **Applicatie detecteert evenement en vraagt gebruiker om input in de console**
6. Gebruiker typt "ja" of "nee"
7. Applicatie stuurt antwoord via `send_responses_streaming()`
8. decision_manager routeert op basis van het antwoord
9. Eindresultaat wordt weergegeven

**Belangrijk Patroon:**
- Gebruik `workflow.run_stream()` voor de eerste iteratie
- Gebruik `workflow.send_responses_streaming(pending_responses)` voor volgende iteraties
- Luister naar `RequestInfoEvent` om te detecteren wanneer menselijke input nodig is
- Luister naar `WorkflowOutputEvent` om de eindresultaten vast te leggen


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability - Human-in-the-Loop)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → confirmation_agent → request_info_executor → <strong>PAUSE</strong> → decision_manager → (depends on user input)</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], 
    should_respond=True
)

# Human-in-the-loop execution pattern
pending_responses: dict[str, str] | None = None
completed = False
workflow_output: str | None = None

print("\n🔄 Starting human-in-the-loop workflow...")
print("=" * 60)

while not completed:
    # First iteration uses run_stream with the request
    # Subsequent iterations use send_responses_streaming with collected human responses
    if pending_responses:
        print(f"\n📤 Sending human responses: {pending_responses}")
        stream = workflow.send_responses_streaming(pending_responses)
        pending_responses = None  # Clear immediately after sending
    else:
        print(f"\n🚀 Starting workflow with request: 'I want to book a hotel in Paris'")
        stream = workflow.run_stream(request_paris)
    
    # Collect all events from this iteration
    events = [event async for event in stream]
    
    # Process events
    requests: list[tuple[str, str]] = []  # (request_id, prompt)
    
    for event in events:
        # Check for human input requests
        if isinstance(event, RequestInfoEvent) and isinstance(event.data, HumanFeedbackRequest):
            print(f"\n⏸️  WORKFLOW PAUSED - Human input requested!")
            print(f"   Request ID: {event.request_id}")
            print(f"   Destination: {event.data.destination}")
            requests.append((event.request_id, event.data.prompt))
        
        # Check for workflow outputs
        elif isinstance(event, WorkflowOutputEvent):
            workflow_output = str(event.data)
            completed = True
            print(f"\n✅ Workflow completed with output!")
    
    # If we have human requests, prompt the user
    if requests and not completed:
        responses: dict[str, str] = {}
        for req_id, prompt in requests:
            print(f"\n{'='*60}")
            print(f"💬 QUESTION FOR YOU:")
            print(f"   {prompt}")
            print(f"{'='*60}")
            
            # Get user input (in notebook, this will pause execution)
            answer = input("👉 Enter 'yes' or 'no': ").strip().lower()
            
            print(f"\n📝 You answered: {answer}")
            responses[req_id] = answer
        
        pending_responses = responses

print(f"\n{'='*60}")
print(f"🏆 FINAL WORKFLOW OUTPUT:")
print(f"{'='*60}")

# Display final result
if workflow_output:
    # Try to parse as JSON for pretty display
    try:
        result_data = json.loads(workflow_output)
        if "alternative_destination" in result_data:
            result_obj = AlternativeResult.model_validate_json(workflow_output)
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> ✅ Accepted alternatives</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_obj.alternative_destination}</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_obj.reason}</p>
                    </div>
                </div>
            """)
            )
        else:
            # User declined
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #f44336 0%, #e91e63 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(244,67,54,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> 🚫 Declined alternatives</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Result:</strong> Booking request cancelled</p>
                    </div>
                </div>
            """)
            )
    except:
        print(workflow_output)


🔄 Starting human-in-the-loop workflow...

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: 032c8fce-b9d1-400e-ba8d-afd2248e2926
   Destination: Paris

💬 QUESTION FOR YOU:
   Unfortunately, there are no rooms available in Paris. Would you like to explore nearby alternative destinations?

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: cf48dad0-ee5e-4f60-8806-341a7a292bd4
   Destination: Paris

💬 QUESTION FOR YOU:
   I'm sorry to inform you that there are no available hotel rooms in Paris. Would you like me to suggest nearby alternative destinations?

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'


## Stap 11: Voer Testgeval 2 uit - Stad MET Beschikbaarheid (Stockholm - Geen Menselijke Input Nodig)

Deze test toont het **directe pad** wanneer kamers beschikbaar zijn:

1. Vraag hotel in Stockholm aan
2. availability_agent controleert → Kamers beschikbaar ✅
3. booking_agent stelt een boeking voor
4. display_result toont bevestiging
5. **Geen menselijke input vereist!**

De workflow omzeilt volledig het pad met menselijke tussenkomst wanneer kamers beschikbaar zijn.


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability - No Human Input)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result (direct, no pause)</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], 
    should_respond=True
)

# Run the workflow (should complete without human input)
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm - No Human Input)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0 0 10px 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <p style='margin: 10px 0 0 0; font-size: 12px; color: #999; font-style: italic;'>Note: No human input was requested because rooms were available!</p>
            </div>
        </div>
    """)
    )

## Belangrijke inzichten en best practices voor mens-in-de-lus

### ✅ Wat je hebt geleerd:

#### 1. **RequestInfoExecutor-patroon**
Het mens-in-de-lus-patroon in Microsoft Agent Framework gebruikt drie belangrijke componenten:
- `RequestInfoExecutor` - Pauzeert de workflow en genereert gebeurtenissen
- `RequestInfoMessage` - Basisklasse voor getypeerde verzoekpayloads (subklasse hiervan!)
- `RequestResponse` - Correlatie tussen menselijke antwoorden en oorspronkelijke verzoeken

**Belangrijke inzichten:**
- `RequestInfoExecutor` verzamelt zelf geen input - het pauzeert alleen de workflow
- Je applicatiecode moet luisteren naar `RequestInfoEvent` en input verzamelen
- Je moet `send_responses_streaming()` aanroepen met een dict die `request_id` koppelt aan het antwoord van de gebruiker

#### 2. **Streaming-uitvoeringspatroon**
```python
# First iteration
stream = workflow.run_stream(initial_request)

# Subsequent iterations (after human input)
stream = workflow.send_responses_streaming(pending_responses)

# Always process events
events = [event async for event in stream]
```

#### 3. **Event-gedreven architectuur**
Luister naar specifieke gebeurtenissen om de workflow te beheren:
- `RequestInfoEvent` - Menselijke input is nodig (workflow gepauzeerd)
- `WorkflowOutputEvent` - Eindresultaat is beschikbaar (workflow voltooid)
- `WorkflowStatusEvent` - Statuswijzigingen (IN_PROGRESS, IDLE_WITH_PENDING_REQUESTS, etc.)

#### 4. **Aangepaste executors met @handler**
De `DecisionManager` laat zien hoe je executors kunt maken die:
- De `@handler`-decorator gebruiken om methoden als workflowstappen bloot te stellen
- Getypeerde berichten ontvangen (bijv. `RequestResponse[HumanFeedbackRequest, str]`)
- Workflows routeren door berichten naar andere executors te sturen
- Toegang hebben tot context via `WorkflowContext`

#### 5. **Conditionele routering met menselijke beslissingen**
Je kunt functievoorwaarden maken die menselijke antwoorden evalueren:
```python
def user_wants_alternatives_condition(message: Any) -> bool:
    response_text = message.agent_run_response.text.lower()
    return response_text == "yes"
```

### 🎯 Toepassingen in de praktijk:

1. **Goedkeuringsworkflows**
   - Verkrijg goedkeuring van een manager voordat je onkostendeclaraties verwerkt
   - Vereis menselijke controle voordat je geautomatiseerde e-mails verzendt
   - Bevestig transacties met hoge waarde voordat ze worden uitgevoerd

2. **Contentmoderatie**
   - Markeer twijfelachtige inhoud voor menselijke controle
   - Vraag moderators om een definitieve beslissing te nemen in grensgevallen
   - Escaleer naar mensen wanneer AI-zekerheid laag is

3. **Klantenservice**
   - Laat AI routinematige vragen automatisch afhandelen
   - Escaleer complexe problemen naar menselijke medewerkers
   - Vraag klanten of ze met een mens willen spreken

4. **Gegevensverwerking**
   - Vraag mensen om onduidelijke gegevensinvoer op te lossen
   - Bevestig AI-interpretaties van onduidelijke documenten
   - Laat gebruikers kiezen tussen meerdere geldige interpretaties

5. **Veiligheidskritieke systemen**
   - Vereis menselijke bevestiging voordat onomkeerbare acties worden uitgevoerd
   - Verkrijg goedkeuring voordat je toegang krijgt tot gevoelige gegevens
   - Bevestig beslissingen in gereguleerde sectoren (gezondheidszorg, financiën)

6. **Interactieve agenten**
   - Bouw conversatiebots die vervolgvragen stellen
   - Maak wizards die gebruikers door complexe processen begeleiden
   - Ontwerp agenten die stap voor stap met mensen samenwerken

### 🔄 Vergelijking: Met vs Zonder mens-in-de-lus

| Kenmerk | Conditionele workflow | Mens-in-de-lus workflow |
|---------|-----------------------|-------------------------|
| **Uitvoering** | Enkel `workflow.run()` | Loop met `run_stream()` + `send_responses_streaming()` |
| **Gebruikersinput** | Geen (volledig geautomatiseerd) | Interactieve prompts via `input()` of UI |
| **Componenten** | Agents + Executors | + RequestInfoExecutor + DecisionManager |
| **Gebeurtenissen** | Alleen AgentExecutorResponse | RequestInfoEvent, WorkflowOutputEvent, etc. |
| **Pauzeren** | Geen pauzeren | Workflow pauzeert bij RequestInfoExecutor |
| **Menselijke controle** | Geen menselijke controle | Mensen nemen belangrijke beslissingen |
| **Gebruiksscenario** | Automatisering | Samenwerking & toezicht |

### 🚀 Geavanceerde patronen:

#### Meerdere menselijke beslissingspunten
Je kunt meerdere `RequestInfoExecutor`-knooppunten in dezelfde workflow hebben:
```python
.add_edge(agent1, request_info_1)  # First human decision
.add_edge(decision_manager_1, agent2)
.add_edge(agent2, request_info_2)  # Second human decision
.add_edge(decision_manager_2, final_agent)
```

#### Timeout-afhandeling
Implementeer time-outs voor menselijke antwoorden:
```python
import asyncio

try:
    answer = await asyncio.wait_for(
        asyncio.to_thread(input, "Enter yes/no: "),
        timeout=60.0
    )
except asyncio.TimeoutError:
    answer = "no"  # Default to safe option
```

#### Integratie met rijke UI
In plaats van `input()`, integreren met web UI, Slack, Teams, etc.:
```python
if isinstance(event, RequestInfoEvent):
    # Send notification to user's preferred channel
    await slack_client.send_message(
        user_id=current_user,
        text=event.data.prompt,
        request_id=event.request_id
    )
```

#### Conditionele mens-in-de-lus
Vraag alleen om menselijke input in specifieke situaties:
```python
def needs_human_approval_condition(message: Any) -> bool:
    # Only route to human if confidence is low or value is high
    if result.confidence < 0.7 or result.value > 10000:
        return True
    return False
```

### ⚠️ Best practices:

1. **Altijd RequestInfoMessage subklassen**
   - Zorgt voor typeveiligheid en validatie
   - Maakt rijke context voor UI-weergave mogelijk
   - Verduidelijkt de intentie van elk type verzoek

2. **Gebruik beschrijvende prompts**
   - Geef context over wat je vraagt
   - Leg de gevolgen van elke keuze uit
   - Houd vragen eenvoudig en duidelijk

3. **Afhandelen van onverwachte input**
   - Valideer gebruikersantwoorden
   - Voorzie standaardwaarden voor ongeldige input
   - Geef duidelijke foutmeldingen

4. **Volg verzoek-ID's**
   - Gebruik de correlatie tussen request_id en antwoorden
   - Probeer geen status handmatig te beheren

5. **Ontwerp voor niet-blokkerend**
   - Blokkeer geen threads terwijl je op input wacht
   - Gebruik asynchrone patronen overal
   - Ondersteun gelijktijdige workflow-instanties

### 📚 Gerelateerde concepten:

- **Agent Middleware** - Intercepteer agentaanroepen (vorige notebook)
- **Workflowstatusbeheer** - Bewaar workflowstatus tussen runs
- **Samenwerking tussen meerdere agenten** - Combineer mens-in-de-lus met agentteams
- **Event-gedreven architecturen** - Bouw reactieve systemen met gebeurtenissen

---

### 🎓 Gefeliciteerd!

Je hebt mens-in-de-lus workflows met Microsoft Agent Framework onder de knie! Je weet nu hoe je:
- ✅ Workflows kunt pauzeren om menselijke input te verzamelen
- ✅ RequestInfoExecutor en RequestInfoMessage kunt gebruiken
- ✅ Streaming-uitvoering met gebeurtenissen kunt afhandelen
- ✅ Aangepaste executors kunt maken met @handler
- ✅ Workflows kunt routeren op basis van menselijke beslissingen
- ✅ Interactieve AI-agenten kunt bouwen die samenwerken met mensen

**Dit is een cruciaal patroon voor het bouwen van betrouwbare, controleerbare AI-systemen!** 🚀



---

**Disclaimer**:  
Dit document is vertaald met behulp van de AI-vertalingsservice [Co-op Translator](https://github.com/Azure/co-op-translator). Hoewel we streven naar nauwkeurigheid, dient u zich ervan bewust te zijn dat geautomatiseerde vertalingen fouten of onnauwkeurigheden kunnen bevatten. Het originele document in de oorspronkelijke taal moet worden beschouwd als de gezaghebbende bron. Voor kritieke informatie wordt professionele menselijke vertaling aanbevolen. Wij zijn niet aansprakelijk voor eventuele misverstanden of verkeerde interpretaties die voortvloeien uit het gebruik van deze vertaling.
